In [20]:
import numpy as np
import math
import numpy.linalg as la
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import quaternion
from quaternion import quaternion
from numpy.linalg import norm
%matplotlib qt

In [21]:
class Particle:
    def __init__(self, pos, m):
        self.x = np.array(pos, dtype=np.float64)
        self.m = m
        self.prevX = None
        self.v = np.zeros(len(self.x))
        self.fExt = np.zeros(len(self.x))
    
    def preUpdate(self, dt):
        self.prevX = np.copy(self.x)
        self.v += (dt / self.m) * self.fExt
        self.x += self.v * dt
    
    def postUpdate(self, dt):
        self.v = (self.x - self.prevX) / dt
    
    def applyForce(self, f, pos):
        self.fExt += f
    
    def draw(self):
        pass

class RigidBody:
    def __init__(self, x, q, m, I, draw=lambda x: None):
        self.x = np.array(x, dtype=np.float64)
        self.set_q(q)    # NOTE: quaternion data type!
        self.m = m
        self.I = I
        self.Iinv = la.inv(I)
        self.prevX = None
        self.prevQ = None
        self.v = np.zeros(3)
        self.w = np.zeros(3)
        self.prevV = None
        self.prevW = None
        self.fExt = np.zeros(3)
        self.TExt = np.zeros(3)
        self._draw = draw
    
    def preUpdate(self, dt):
        self.prevX = np.copy(self.x)
        self.prevV = np.copy(self.v)
        self.v += (dt / self.m) * self.fExt
        self.x += self.v * dt
                
        self.prevQ = self.q
        self.prevW = np.copy(self.w)
        # TODO: Do the math derivations. This is copied from the paper.
        self.w += dt * (self.Iinv @ (self.TExt - np.cross(self.w, self.I @ self.w)))
        self.set_q((self.q + (dt / 2 * (quaternion(0, *self.w) * self.q))).normalized())
    
    def reset(self):
        self.q = self.prevQ
        self.w = self.prevW
        self.x = self.prevX
        self.v = self.prevV
    
    def postUpdate(self, dt):
        self.v = (self.x - self.prevX) / dt
        
        # Again, take some time to understand this math. I mean take more time.
        dq = self.q * self.prevQ.reciprocal()
        # For example, why is there a signum here? Some double cover stuff going weird
        self.w = 2*dq.vec * np.sign(dq.real) / dt
    
    def draw(self):
        self._draw(self)
    
    def set_q(self, q_new):
        self.q = q_new
        self.qinv = q_new.reciprocal()

    def localToWorld_F(self, vec):
        """
        Convert between frames.
        """
        vq = quaternion(0, *vec)
        return self.x + (self.q * vq * self.qinv).vec
    
    def worldToLocal_F(self, vec):
        vq = quaternion(0, *vec)
        return (self.qinv * vq * self.q).vec - self.x

    def localToWorld_R(self, vec):
        """
        Convert between orientations only.
        """
        vq = quaternion(0, *vec)
        return (self.q * vq * self.qinv).vec
    
    def worldToLocal_R(self, vec):
        vq = quaternion(0, *vec)
        return (self.qinv * vq * self.q).vec

    
class FixConstraint:
    def __init__(self, node):
        self.x = np.copy(node.x)
        self.node = node
        self.y = None
        self.f = None
        self.fdir = None
    
    def preUpdate(self, dt):
        self.y = 0
        self.f = None
        self.fdir = None
    
    def apply(self):
        err = norm(self.node.x - self.x)
        self.y += err
        self.fdir = (self.node.x - self.x) / err
        self.node.x = np.copy(self.x)
    
    def postUpdate(self, dt):
        self.f = self.y * self.fdir / dt**2
        
    def draw(self):
        plt.gca().add_patch(patches.Circle(self.node.x[0:2], radius=0.1,ec='black',fill=False))
        
class PositionBodyConstraint:
    def __init__(self, node1, node2, r1, r2, d, compliance=0):
        """
        Parameters:
         - node1: First body
         - node2: Second body
         - r1: Position on first body, body frame
         - r2: Position on second body, body frame
         - d: Distance to maintain
         - compliance: compliance
        """
        self.node1 = node1
        self.node2 = node2
        self.d = d
        self.r1 = np.array(r1, dtype=np.float64)
        self.r2 = np.array(r2, dtype=np.float64)
        self.compliance = compliance
        
        self.fix1 = False
        self.fix2 = False
        if node1.m == np.inf:
            self.fix1 = True
        if node2.m == np.inf:
            self.fix2 = True
        if self.fix1 and self.fix2:
            raise ValueError("Can't have both nodes in a joint have infinite mass!")
            
        self.y = None
        self.f = None
        self.fdir = None
        self.rErr = None
        self.aErr = None
    
    def preUpdate(self, dt):
        self.y = 0
        self.alpha = self.compliance / dt**2
        self.f = None
        self.fdir = np.zeros(3)
        self.rErr = 0
        self.aErr = 0
    
    def apply(self):
        dispvec = self.node2.localToWorld_F(self.r2) - self.node1.localToWorld_F(self.r1)
        
        if norm(dispvec) == 0:
            self.rErr = 0
            self.aErr = 0
            return;
        
        dispvec_mag = norm(dispvec)
        
        err = dispvec_mag - self.d
        n = dispvec / dispvec_mag
        delta = n * err
        arm1 = np.cross(self.r1, self.node1.worldToLocal_R(n))
        arm2 = np.cross(self.r2, self.node2.worldToLocal_R(n))
        
        w1 = 1/self.node1.m + np.dot(arm1, self.node1.Iinv @ arm1)
        w2 = 1/self.node2.m + np.dot(arm2, self.node2.Iinv @ arm2)
        if self.fix1:
            w1 = 0
        elif self.fix2:
            w2 = 0
        
        dy = (err - self.alpha*self.y) / (w1 + w2 + self.alpha)
        self.y += dy
        
        p = dy*n
        
        if not self.fix1:
            self.node1.x += p / self.node1.m
            new_q1 = (self.node1.q + dy*0.5*quaternion(0, *(self.node1.Iinv @ arm1))*self.node1.q)#.normalized()
            self.node1.set_q(new_q1)
        if not self.fix2:
            self.node2.x -= p / self.node2.m
            new_q2 = (self.node2.q - dy*0.5*quaternion(0, *(self.node2.Iinv @ arm2))*self.node2.q)#.normalized()
            self.node2.set_q(new_q2)
        
        self.fdir = n
        
        if self.d != 0:
            self.rErr = dy / self.d
        self.aErr = dy
    
    def err(self):
        return abs(norm(self.node2.localToWorld_F(self.r2) - self.node1.localToWorld_F(self.r1)) - self.d)
    
    def postUpdate(self, dt):
        self.f = self.y * self.fdir / dt**2
    
    def draw(self):
        x1 = self.node1.x[:2]
        x2 = self.node1.localToWorld_F(self.r1)[:2]
        x3 = self.node2.localToWorld_F(self.r2)[:2]
        x4 = self.node2.x[:2]
        plt.plot(*zip(*(x1, x2)),linestyle='--',color='black')
        plt.plot(*zip(*(x2, x3)),linestyle='--',color='red')
        plt.plot(*zip(*(x3, x4)),linestyle='--',color='black')
        
class PositionConstraint:
    def __init__(self, node1, node2, r):
        self.node1 = node1
        self.node2 = node2
        self.r = r
        m1 = node1.m
        m2 = node2.m
        if m1 == np.inf:
            self.r1 = 0
            self.r2 = 1
        elif m2 == np.inf:
            self.r1 = 1
            self.r2 = 0
        else:
            mt = m1 + m2
            self.r1 = m2/mt
            self.r2 = m1/mt
        self.y = None
        self.f = None
    
    def preUpdate(self, dt):
        self.y = 0
        self.f = None
    
    def apply(self):
        posvec = self.node2.x - self.node1.x
        delta = posvec * (norm(posvec) - self.r)
        self.node1.x += delta * self.r1
        self.node2.x -= delta * self.r2
        self.y += norm(delta)
    
    def postUpdate(self, dt):
        posvec = self.node2.x - self.node1.x
        self.f = self.y * posvec / (norm(posvec)*dt**2)
    
    def draw(self):
        plt.plot(*zip(*(self.node1.x[:2], self.node2.x[:2])),linestyle='--',color='black')

In [22]:
def setup_fig(n=-1):
    if n == -1:
        plt.figure()
    else:
        plt.figure(n)
    
def draw(particles, constraints, box):
    plt.clf()
    plt.gca().set_aspect(aspect="equal",adjustable="box")
    plt.xlim(box[0])
    plt.ylim(box[1])
    for constraint in constraints:
        constraint.draw()
    for particle in particles:
        particle.draw()
    plt.scatter(*zip(*[node.x[:2] for node in particles]))

def try_update(particles, constraints, dt, rtol=1e-12, atol=1e-6):
    for constraint in constraints:
        constraint.preUpdate(dt)
    for particle in particles:
        particle.preUpdate(dt)
    
    for constraint in constraints:
        constraint.apply()
    for constraint in constraints:
        err = constraint.err()
        if err > atol or (constraint.d != 0 and err / constraint.d > rtol):
            for particle in particles:
                particle.reset()
            return False
    
    for particle in particles:
        particle.postUpdate(dt)
    for constraint in constraints:
        constraint.postUpdate(dt)
    return True

    
# def update_converge(particles, constraints, dt, rtol=1e-24, atol=1e-12):
def update_converge(particles, constraints, dt, rtol=1e-12, atol=1e-6):
    for constraint in constraints:
        constraint.preUpdate(dt)
    for particle in particles:
        particle.preUpdate(dt)
    
    err_pass = False
    niter = 0
    while not err_pass:
        err_pass = True
        for constraint in constraints:
            constraint.apply()
        for constraint in constraints:
            err = constraint.err()
            if err > atol:
                err_pass = False
            elif constraint.d != 0 and err / constraint.d > rtol:
                err_pass = False
        niter += 1
    for particle in particles:
        particle.postUpdate(dt)
    for constraint in constraints:
        constraint.postUpdate(dt)
    print(f"Converged in {niter} iterations")

def update(particles, constraints, dt, niter=1):
    for constraint in constraints:
        constraint.preUpdate(dt)
    for particle in particles:
        particle.preUpdate(dt)
    for i in range(niter):
        for constraint in constraints:
            constraint.apply()
    for particle in particles:
        particle.postUpdate(dt)
    for constraint in constraints:
        constraint.postUpdate(dt)

def angle_quat(angle):
    return quaternion(np.cos(angle/2), 0, 0, np.sin(angle/2))

def rotate(vec, quat):
    vq = quaternion(0, *vec)
    return (quat * vq * quat.reciprocal()).vec
        
def IzMat(I):
    return np.array([[1000*I, 0, 0], [0, 1000*I, 0], [0, 0, I]])

In [23]:
def makeCrank2D(x, q, l, w, t, rho):
    """
    l is dimension in x, w is dimension in y.
    """
    v = l*w*t
    m = v*rho
    I = m*(l**2+w**2)/12
    vec1 = np.array([l/2, w/2, 0])
    vec2 = np.array([l/2, -w/2, 0])
    def _draw(self):
        tvec1 = self.localToWorld_R(vec1)
        tvec2 = self.localToWorld_R(vec2)
        box_pos = list(zip(self.x + tvec1, self.x - tvec2, self.x - tvec1, self.x + tvec2, self.x+tvec1))[:2]
        plt.plot(box_pos[0], box_pos[1], color='orange')
    print(f"m={m} I={I}")
    return RigidBody(x, q, m, IzMat(I), draw=_draw)

In [81]:
def setup1():
    center = RigidBody((0, 0, 0), angle_quat(0), np.inf, IzMat(1))
    mid = RigidBody((0.5, 0, 0), angle_quat(0), 1, IzMat(1))
    side = RigidBody((1, 0, 0), angle_quat(0), 1, IzMat(1))

    #side = Particle((0, 1, 0), 1)
    #radius = PositionConstraint(center, side, 1)
    swing1 = PositionBodyConstraint(mid, side, (0, 0, 0), (-0.5, 0, 0), 0, 0)
    swing2 = PositionBodyConstraint(center, mid, (0, 0, 0), (-0.5, 0, 0), 0, 0)
    
    constraints = [swing1, swing2]
    particles = [center, mid, side]
    
    box = ((-2, 2), (-2, 2))
    setup_fig(1)

    side.fExt = np.array([0, 1, 0])
    
    return constraints, particles, bpx

def setup2():
    ground = RigidBody((0, 0, 0), angle_quat(0), np.inf, IzMat(1))
    
    l1 = RigidBody((-0.1, 0.05, 0), angle_quat(np.pi/2), 1, IzMat(1))
    l2 = RigidBody((0.1, 0.125, 0), angle_quat(np.pi/2), 2.5, IzMat(2.5**2))
    l3 = RigidBody((0, 0.175, 0), angle_quat(0), 2.5, IzMat(2.5**2))

    cg1 = PositionBodyConstraint(ground, l1, (-0.1, 0, 0), (-0.05, 0, 0), 0, 0)
    cg2 = PositionBodyConstraint(ground, l2, (0.1, 0, 0), (-0.125, 0, 0), 0, 0)
    c13 = PositionBodyConstraint(l1, l3, (0.05, 0, 0), (-0.1, -0.075, 0), 0, 0)
    c23 = PositionBodyConstraint(l2, l3, (0.125, 0, 0), (0.1, 0.075, 0), 0, 0)
    
    l1.v = np.array([-0.25, 0, 0])
    l1.w = np.array([0, 0, 0.5])
    l2.v = np.array([-0.125, 0, 0])
    l2.w = np.array([0, 0, 0.25])
    l3.v = np.array([-0.5, 0, 0])

    constraints = [cg1, cg2, c13, c23]
    particles = [ground, l1, l2, l3]
    
    box = ((-0.25, 0.15), (-0.125, 0.275))
    setup_fig(1)

    return constraints, particles, box

def setup3():
    ground = RigidBody((0, 0, 0), angle_quat(0), np.inf, IzMat(1))
    
    angle2 = math.acos(-8/40)
    angle3 = math.acos(8/40)
    h = math.sqrt(40**2-8**2)/1000
    
    acrylic_density=1190
    l1 = makeCrank2D((8/1000, 0, 0), angle_quat(0), 16/1000, 0.0127, 0.00635, acrylic_density)
    l2 = makeCrank2D((28/1000, h/2, 0), angle_quat(angle2), 40/1000, 0.0127, 0.00635, acrylic_density)
    l3 = makeCrank2D((24/1000, h, 0), angle_quat(angle3), 80/1000, 0.0127, 0.00635, acrylic_density)
    
    cg1 = PositionBodyConstraint(ground, l1, (0, 0, 0), (-8/1000, 0, 0), 0, 0)
    cg2 = PositionBodyConstraint(ground, l2, (32/1000, 0, 0), (-20/1000, 0, 0), 0, 0)
    c13 = PositionBodyConstraint(l1, l3, (8/1000, 0, 0), (-40/1000, 0, 0), 0, 0)
    c23 = PositionBodyConstraint(l2, l3, (20/1000, 0, 0), (0, 0, 0), 0, 0)
    
    constraints = [cg1, cg2, c13, c23]
#     constraints = [c13]
    particles = [ground, l1, l2, l3]
    
    box = ((-0.05, 0.1), (-0.05, 0.1))
    setup_fig(1)

    return constraints, particles, box

def setup4():
#     p0 = Vector(0,0)
#     p1 = Vector(5.0,0.0)
#     p2 = Vector(4.149,3.908)
#     p3 = Vector(1.294,4.830)
#     self.pneuAttachVector = Vector(1, 1.732)
#     self.pneuPivot = Vector(-4.0,0.25)
#     self.cGVectorOffset = radians(12)
#     self.cGVector = Vector(5.0, self.l2.vector.theta + self.cGVectorOffset, True)
#     self.loadVector = Vector(0.0, -10.0)
    
    p0 = np.array((0.0,0.0,0.0))
    p1 = np.array((5.0,0.0,0.0))
    p2 = np.array((4.149,3.908,0.0))
    p3 = np.array((1.294,4.830,0.0))
    pneumaticPivot = np.array((-4, 0.25, 0))
    pneumaticAttach = np.array((1, 1.732, 0))
    v2 = p2 - p3
    v2hat = v2 / norm(v2)
    cgAngle = math.radians(12)
    cgVec = 5 * rotate(v2hat, angle_quat(cgAngle))
    v2Rest = v2 - cgVec
    v3c = (p0+p3)/2
    l3ToPneumatic = pneumaticAttach - v3c

    ground = RigidBody((0, 0, 0), angle_quat(0), np.inf, IzMat(1))
    x3 = norm(p3 - p0)
    l3 = RigidBody(v3c, angle_quat(0), x3, IzMat(x3**2))
    x2 = norm(v2)
    l2 = RigidBody(p3 + cgVec, angle_quat(0), x2, IzMat(x2**2))
    dx1 = p2 - p1
    x1 = norm(dx1)
    l1 = RigidBody((p2+p1)/2, angle_quat(math.atan2(dx1[1], dx1[0])), x1, IzMat(x1**2))
    
    cg3 = PositionBodyConstraint(ground, l3, p0, -v3c, 0, 0)
    cg1 = PositionBodyConstraint(ground, l1, p1, (-x1/2, 0, 0), 0, 0)
    c12 = PositionBodyConstraint(l1, l2, (x1/2, 0, 0), v2Rest, 0, 0)
    c23 = PositionBodyConstraint(l2, l3, -cgVec, v3c, 0, 0)
    
    pneumaticLen = norm(pneumaticPivot - pneumaticAttach)
    pneumatic = PositionBodyConstraint(ground, l3, pneumaticPivot, l3ToPneumatic, pneumaticLen, 0)
    
    l2.fExt = np.array((0, -10, 0))
    
    constraints = [cg1, cg3, c12, c23, pneumatic]
    particles = [ground, l1, l2, l3]
    
    box = ((-4.5, 6.5), (-1, 10))
    setup_fig(1)
    print(l3ToPneumatic)
    return constraints, particles, box, pneumatic


constraints, particles, box, pneumatic = setup4()

draw(particles, constraints, box)
plt.show()
plt.pause(0.01)

[ 0.353 -0.683  0.   ]


In [87]:
plt.ion()

import threading
import time
import math

dt = 1/40/600
# n_substeps = 2*6
n_substeps = 200
should_plot = False
sim_done = False

tf = 0.05
# tf=400*dt
# particles[1].TExt = np.array((0, 0, 0.1))
# particles[1].TExt

nframe=int(tf/n_substeps/dt)

# update(particles, constraints, dt)

def _sim_thread():
    global should_plot
    global particles
    global constraints
    global box
    global dt
    global n_substeps
    global sim_done
    global nframe
    global tf
    global fxs
    global fys
    global ts
    global energy
    fxs = []
    fys = []
    energy = []
#     angles = []
    ts = []
    t = 0
    cur_dt = dt
    tframe = dt * n_substeps
    drawT = 0
    while t < tf:
        update(particles, constraints, cur_dt)
        
        step_fx = []
        step_fy = []
        for c in constraints:
            step_fx.append(c.f[0])
            step_fy.append(c.f[1])
        fxs.append(step_fx)
        fys.append(step_fy)

        e = 0
        for p in particles:
            if p.m != np.inf:
                e += 0.5 * (p.m * np.dot(p.v, p.v) + np.dot(p.w, p.I @ p.w))
        energy.append(e)

#         angles.append(math.atan2(particles[1].x[1], particles[1].x[0]+0.1))
        ts.append(t)
        
        t += cur_dt
        
        if t - drawT > tframe:
            should_plot = True
            drawT += tframe
    
    fxs = list(zip(*fxs))
    fys = list(zip(*fys))
    
    time.sleep(1)
    
    plt.figure(2)
    plt.clf()
    for dat in fxs:
        plt.plot(ts, dat)
    plt.legend([str(i) for i in range(len(fxs))])
    plt.title("X force over time")
    
    plt.figure(3)
    plt.clf()
    for dat in fys:
        plt.plot(ts, dat)
    plt.legend([str(i) for i in range(len(fys))])
    plt.title("Y force over time")
    
    plt.figure(4)
    plt.clf()
    plt.plot(ts, energy)
    plt.title("Energy over time")
    
#     plt.figure(5)
#     plt.clf()
#     plt.plot(ts, angles)
#     plt.title("Crank angle over time")
    
    plt.show()
    plt.pause(0.01)
    plt.figure(1)
    
    time.sleep(0.2)
    sim_done = True

#_sim_thread()
sim_thread = threading.Thread(target=_sim_thread)
sim_thread.start()

while True:
    if should_plot:
        should_plot = False
        draw(particles, constraints, box)
        plt.show()
        plt.pause(0.01)
    else:
        time.sleep(0.05)
    if sim_done:
        break

sim_thread.join()
print("Sim done!")

Sim done!


In [63]:
print(particles[1].x)
print(particles[2].x)
print(particles[3].x)

[3.02562411 1.08046466 0.        ]
[3.19250693 2.17787993 0.        ]
[-0.92289694  2.10062672  0.        ]


In [88]:
fnet = np.array((sum(fxs[4]), sum(fys[4])))/len(fxs[4])
print(fnet)
print(norm(fnet))

[-8.36828051 -1.89033002]
8.579129695404003


In [48]:
print(pneumatic.d)

5.215009491841793


In [84]:
# pneumatic.d = 5.215
pneumatic.d = 5.5